In [1]:
from pathlib import Path
import os
import sys
import gc
import shutil
import json
import math
from collections import defaultdict
import numpy as np
import pandas as pd
import bitsandbytes
import accelerate
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from tqdm import tqdm
import matplotlib.pyplot as plt
from typing import Dict, List, Tuple, NamedTuple, Callable, Iterable, Set, Optional, Any
import scml
from mylib.datageneration import Prompter
print(f"accelerate={accelerate.__version__}, bitsandbytes={bitsandbytes.__version__}")

accelerate=0.29.2, bitsandbytes=0.43.1


In [2]:
target_size = 30*6
model_dir = Path("/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1")
model_max_length = 4096
seed = 422

In [3]:
tim = scml.Timer()
tim.start()
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
os.environ["TOKENIZERS_PARALLELISM"] = "false"
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything(seed)
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")
    for i in range(torch.cuda.device_count()):
        print(f"device={i}, {torch.cuda.get_device_name(i)}")
        print('Mem Allocated:', round(torch.cuda.memory_allocated(i)/1024**3,1), 'GB')
        print('Mem Cached:   ', round(torch.cuda.memory_reserved(i)/1024**3,1), 'GB')
else:
    print("cpu")

device=0, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB
device=1, NVIDIA GeForce RTX 4070 Ti SUPER
Mem Allocated: 0.0 GB
Mem Cached:    0.0 GB


In [4]:
with open("input/student_names.json") as f:
    student_names = set(json.load(f))
print(f"{len(student_names):,} student names")

104 student names


In [5]:
df = pd.read_csv("input/geocode-db.csv", low_memory=False, names=["street","lat","lng"])
street_addresses = [x.title() for x in df["street"]]
print(f"{len(street_addresses):,} street addresses={street_addresses[:10]}")
street_addresses = set(street_addresses)
df.info()

11,562 street addresses=['Adis Road (Liv On Sophia)', 'Adis Road (Parc Sophia)', 'Admiralty Drive (Block 353A)', 'Admiralty Drive (Block 353B)', 'Admiralty Drive (Block 353C)', 'Admiralty Drive (Block 354A)', 'Admiralty Drive (Block 354C)', 'Admiralty Drive (Block 354D)', 'Admiralty Drive (Block 356B)', 'Admiralty Drive (Block 356C)']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11562 entries, 0 to 11561
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   street  11562 non-null  object 
 1   lat     11562 non-null  float64
 2   lng     11562 non-null  float64
dtypes: float64(2), object(1)
memory usage: 271.1+ KB


In [6]:
tokenizer = AutoTokenizer.from_pretrained(str(model_dir), model_max_length=model_max_length)
tokenizer.pad_token = tokenizer.eos_token
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")
print(f"pad_token_id={tokenizer.pad_token_id}")

LlamaTokenizerFast(name_or_path='/mnt/c/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1', vocab_size=32000, model_max_length=4096, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'attention_mask']
pad_token_id=2


In [7]:
%%time
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
    str(model_dir),
    device_map = "auto",
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

CPU times: user 4min 9s, sys: 3min 34s, total: 7min 43s
Wall time: 14min 1s


In [8]:
print(model)

MixtralForCausalLM(
  (model): MixtralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MixtralDecoderLayer(
        (self_attn): MixtralSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MixtralRotaryEmbedding()
        )
        (block_sparse_moe): MixtralSparseMoeBlock(
          (gate): Linear4bit(in_features=4096, out_features=8, bias=False)
          (experts): ModuleList(
            (0-7): 8 x MixtralBlockSparseTop2MLP(
              (w1): Linear4bit(in_features=4096, out_features=14336, bias=False)
              (w2): Linear4bit(in_features=14336, out_features=4096, bias=False)
              (w3): Linear4bit(in_features=4096, out_

In [9]:
prompter = Prompter(target_size=target_size, student_names=student_names, street_addresses=street_addresses)
res = prompter.prompt()
print(res.prompt)

[INST] Write a short essay on the following topic:
        In Education Policy and Reform, how can the integration of a learning launch tool within the design thinking process enhance the development and testing of ideas, prototypes, and products?
        Intersperse the following information in different parts of the essay:
        1. Give an example that occurred near Pasir Ris Street 11 (Block 125), at a school frequented by Justin Kaiser
        2. Give an example citing someone named JustinKaiser
        3. Give an example citing: periscope.tv/officialjustiniamkaiser [/INST]
        


In [10]:
%%time
rows = []
for _ in tqdm(range(target_size)):
    res = prompter.prompt()
    inputs = tokenizer(res.prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs, 
        max_new_tokens=512,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        top_k=100,
        repetition_penalty=1.1,
        pad_token_id=tokenizer.eos_token_id,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    generated_text = generated_text.split("[/INST]")[1]
    generated_text = generated_text.strip()
    rows.append({
        "text": generated_text,
        "prompt": res.prompt,
        "street_address": res.street_address,
        "student_name": res.student_name,
        "username": res.username,
        "personal_url": res.personal_url,
        "model": model_dir.name,
    })

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 180/180 [5:15:41<00:00, 105.23s/it]

CPU times: user 4h 57min 30s, sys: 18min 13s, total: 5h 15min 44s
Wall time: 5h 15min 41s


In [11]:
df = pd.DataFrame.from_records(rows)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   text            180 non-null    object
 1   prompt          180 non-null    object
 2   street_address  180 non-null    object
 3   student_name    180 non-null    object
 4   username        180 non-null    object
 5   personal_url    180 non-null    object
 6   model           180 non-null    object
dtypes: object(7)
memory usage: 10.0+ KB


In [12]:
df.head(10)

,text,prompt,street_address,student_name,username,personal_url,model
0,"In the field of architecture and urban design, visualization tools have become increasingly important for fostering innovation and collaboration in a design thinking approach. These tools allow designers to communicate their ideas more effectively, enabling stakeholders to better understand the proposed designs and provide valuable feedback. This collaborative process leads to innovative solutions that are tailored to the needs of the community.\n\n One example of this can be seen near Serangoon North, at a nature park frequented by Ahmad bin Abdullah. A group of architects and urban planners used visualization tools to gather input from the community regarding the redesign of the park. By creating 3D models and virtual reality simulations, they were able to demonstrate the potential impact of their plans on the environment and the community. Through this process, Ahmad bin Abdullah was able to share his insights on how the park could better serve the community's needs. His suggestions, combined with those of other stakeholders, led to a final design that improved accessibility and safety while preserving the natural beauty of the space.\n\n The use of visualization tools can also enable individuals, such as 4807.ahmad-iam--abdullah, to contribute to the design process. For instance, if 4807.ahmad-iam--abdullah had an idea for improving the layout of a building or public space, he could easily create a visual representation using free or low-cost software. This user-generated content can then be shared with architects and urban planners, promoting collaboration and ensuring that diverse perspectives are considered throughout the design process.\n\n Another powerful example comes from periscope.tv/ahmadabdullahxxx, where Ahmad Abdullah live-streamed a presentation on a new urban design concept. Using real-time visualization tools, he demonstrated the feasibility of his idea and the benefits it would bring to the community. As viewers provided feedback and suggested modifications, Ahmad Abdullah was able to refine his concept and present a more robust proposal that met the community's needs while showcasing his creative vision.\n\n Visualization tools not only facilitate communication between designers and the community, but they also help designers explore various options and assess their viability quickly. By generating realistic representations of proposed designs, these tools enable designers to identify potential issues early in the process and make adjustments accordingly. This iterative approach ultimately leads to better designs","[INST] Write a short essay on the following topic:\n In Architecture and Urban Design, how can visualization tools foster innovation and collaboration in a design thinking approach?\n Intersperse the following information in different parts of the essay:\n 1. Give an example that occurred near Serangoon North (Block 105), at a nature park frequented by Ahmad bin Abdullah\n 2. Give an example citing someone named _4807.ahmad-iam--abdullah\n 3. Give an example citing: periscope.tv/ahmadabdullahxxx [/INST]\n",Serangoon North (Block 105),Ahmad bin Abdullah,_4807.ahmad-iam--abdullah,periscope.tv/ahmadabdullahxxx,Mixtral-8x7B-Instruct-v0.1
1,"Sustainable urban development is a critical issue facing cities around the world today. As urban populations continue to grow, it is essential that we find innovative and collaborative solutions to create sustainable and livable communities. One approach that has shown promise in fostering innovation and collaboration in sustainable urban development is the use of storytelling within the design thinking process.\n\nStorytelling is a powerful tool for inspiring empathy, understanding, and creativity. By sharing stories about the experiences and needs of individuals and communities, designers and planners can gain insights into the complex challenges facing urban areas. These stories can then 

In [13]:
df.to_parquet("output/part.parquet")
assert df.notna().all(axis=None)

In [14]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 5:29:39.276978
